In [1]:
import jf
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits

from marvin.tools import Maps

with fits.open('./data/galaxies_sorted.fits') as hdulist:
    plateifu_list = hdulist[1].data['plateifu']
    
with fits.open('./data/angles.fits') as hdulist:
    angle_list = hdulist[1].data['col2']

angle_list[0] = -160
angle_list[11] = 110
angle_list[10] = 125
angle_list[12] = -70
angle_list[13] = -140

import warnings 
warnings.filterwarnings("ignore")

[INFO]: No release version set. Setting default to DR15


In [2]:
t= jf.stat_list(plateifu_list, angle_list)
t

plateifu,angle,lead_area,trail_area,lead_mean,trail_mean,lead_median,trail_median,sfr,used_area,total_area,pix_used_ratio,asymmetry
str10,int16,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
8944-6101,-160,100.0,102.0,0.016043712127899065,0.016419161676521535,0.015343508434849502,0.013025004368546311,0.8265495811808903,203.0,203.0,1.0,0.33549548229323967
8934-3701,40,170.0,145.0,0.030190967637335503,0.012056494741066489,0.01900231702771389,0.006560418060606447,1.7338975823121356,316.0,321.0,0.9844236760124611,0.595588549730473
8932-12701,-25,1329.0,1233.0,0.00343085400048465,0.002085344416007612,0.0029081954788505145,0.0013371742264779514,1.7867473840753547,2563.0,2775.0,0.9236036036036036,0.4392495638475043
9876-3702,-120,169.0,170.0,0.020966827758221968,0.014856238178247628,0.011822060984684109,0.011027006887247151,1.5353935671810297,340.0,340.0,1.0,0.2505285587585656
8932-3701,-90,253.0,253.0,0.02576671125420227,0.017620925134532728,0.024739356930600413,0.015093824412919114,2.7562777277586616,507.0,507.0,1.0,0.23247670468400086
8950-12705,-20,166.0,281.0,0.008034741572961598,0.005655389018766711,0.007700802734302738,0.004832645414756609,0.7336151454081377,448.0,602.0,0.7441860465116279,0.31432918120713554
8950-1901,-130,71.0,72.0,0.030106731183309733,0.02722702526638794,0.029925535477157723,0.024070776044193692,1.0406234990616716,144.0,144.0,1.0,0.17203766801110146
8931-3703,10,163.0,175.0,0.0037294020262703307,0.0034562616209762457,0.0020729706006103535,0.0024541123540810154,0.3071013908988194,339.0,354.0,0.9576271186440678,0.18063298339708275
9876-3703,180,235.0,244.0,0.0023075485058418983,0.0019266248687298078,0.0010653567573058974,0.0009051068104835503,0.25588288569595496,480.0,490.0,0.9795918367346939,0.2554289127280133


In [3]:
t.write('./stat/stat_sfr_5_18.fits', format='fits')